### Credit:
[Z-Image Github](https://github.com/Tongyi-MAI/Z-Image) <br>
Colab Code: [camenduru](https://github.com/camenduru/Z-Image-jupyter)


In [ ]:
#@title Install Z-Image-Turbo
#ComfyUI Installation
%cd /content/
!git clone https://github.com/comfyanonymous/ComfyUI

%cd /content/ComfyUI
!wget https://raw.githubusercontent.com/NeuralFalconYT/Z-Image-Colab/refs/heads/main/app.py

!pip install -r requirements.txt

#Model Download

!apt -y install -qq aria2

!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/T5B/Z-Image-Turbo-FP8/resolve/main/z-image-turbo-fp8-e4m3fn.safetensors -d /content/ComfyUI/models/diffusion_models -o z-image-turbo-fp8-e4m3fn.safetensors
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/Comfy-Org/z_image_turbo/resolve/main/split_files/text_encoders/qwen_3_4b.safetensors -d /content/ComfyUI/models/clip -o qwen_3_4b.safetensors
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/Comfy-Org/z_image_turbo/resolve/main/split_files/vae/ae.safetensors -d /content/ComfyUI/models/vae -o ae.safetensors

#Clear Terminal
from IPython.display import clear_output
clear_output()
print("\033[92mZ-Image-Turbo Installation successful\033[0m")
import os

paths = [
    "/content/ComfyUI/models/diffusion_models/z-image-turbo-fp8-e4m3fn.safetensors",
    "/content/ComfyUI/models/clip/qwen_3_4b.safetensors",
    "/content/ComfyUI/models/vae/ae.safetensors",
]

for p in paths:
    if not os.path.exists(p):
        print(f"\033[91mMISSING (possibly HuggingFace blocked download): {p}\033[0m")

#@title Keep Alive for Mobile Users
from IPython.display import Audio,display
import numpy as np
display(Audio(np.array([0] * 2 * 3600 * 3000, dtype=np.int8), normalize=False, rate=3000, autoplay=True))


In [ ]:
#@title Gradio app (Might crash on low GPU memory)
%cd /content/ComfyUI
!python app.py

# Run from colab cell

In [ ]:
#@title Utils Code
%cd /content/ComfyUI

import os, random, time

import torch
import numpy as np
from PIL import Image
import re, uuid
from nodes import NODE_CLASS_MAPPINGS

UNETLoader = NODE_CLASS_MAPPINGS["UNETLoader"]()
CLIPLoader = NODE_CLASS_MAPPINGS["CLIPLoader"]()
VAELoader = NODE_CLASS_MAPPINGS["VAELoader"]()
CLIPTextEncode = NODE_CLASS_MAPPINGS["CLIPTextEncode"]()
KSampler = NODE_CLASS_MAPPINGS["KSampler"]()
VAEDecode = NODE_CLASS_MAPPINGS["VAEDecode"]()
EmptyLatentImage = NODE_CLASS_MAPPINGS["EmptyLatentImage"]()

with torch.inference_mode():
    unet = UNETLoader.load_unet("z-image-turbo-fp8-e4m3fn.safetensors", "fp8_e4m3fn_fast")[0]
    clip = CLIPLoader.load_clip("qwen_3_4b.safetensors", type="lumina2")[0]
    vae = VAELoader.load_vae("ae.safetensors")[0]

save_dir="./results"
os.makedirs(save_dir, exist_ok=True)
def get_save_path(prompt):
  save_dir = "./results"
  safe_prompt = re.sub(r'[^a-zA-Z0-9_-]', '_', prompt)[:25]
  uid = uuid.uuid4().hex[:6]
  filename = f"{safe_prompt}_{uid}.png"
  path = os.path.join(save_dir, filename)
  return path

@torch.inference_mode()
def generate(input):
    values = input["input"]
    positive_prompt = values['positive_prompt']
    negative_prompt = values['negative_prompt']
    seed = values['seed'] # 0
    steps = values['steps'] # 9
    cfg = values['cfg'] # 1.0
    sampler_name = values['sampler_name'] # euler
    scheduler = values['scheduler'] # simple
    denoise = values['denoise'] # 1.0
    width = values['width'] # 1024
    height = values['height'] # 1024
    batch_size = values['batch_size'] # 1.0

    if seed == 0:
        random.seed(int(time.time()))
        seed = random.randint(0, 18446744073709551615)

    positive = CLIPTextEncode.encode(clip, positive_prompt)[0]
    negative = CLIPTextEncode.encode(clip, negative_prompt)[0]
    latent_image = EmptyLatentImage.generate(width, height, batch_size=batch_size)[0]
    samples = KSampler.sample(unet, seed, steps, cfg, sampler_name, scheduler, positive, negative, latent_image, denoise=denoise)[0]
    decoded = VAEDecode.decode(vae, samples)[0].detach()
    save_path=get_save_path(positive_prompt)
    Image.fromarray(np.array(decoded*255, dtype=np.uint8)[0]).save(save_path)
    return save_path,seed

## Prompt Generation Tip
Use ```ChatGPT``` or ```Gemini``` to generate positive and negative prompts based on what you want to create.


In [ ]:
positive_prompt = 'A photorealistic, ultra detailed, humorous scene on a bustling dutch street market. A tabby cat anthropomorphized, running on two legs while tightly hugging a large, shiny silver fish. The cat has a determined, dramatic facial expression with wide eyes and an open mouth, as if mid-shout. Behind the cat, a shocked fish vendor in an apron is chasing after it, yelling. Fresh fishes are laid out on a market stall to the right, displayed on ice. The background features open stalls, market signs, and a few bystanders reacting in surprise. Dynamic lighting, rich details, cinematic composition, freeze-frame action shot, expressive motion blur, '  # @param {type: "string"}
negative_prompt = 'low resolution, blurry subject, muddy focus, extreme motion blur, double exposure, ghosting, smearing, pixelated, jpeg artifacts, noise, banding  bad anatomy, deformed cat, extra legs, missing legs, extra tail, broken spine, twisted body, malformed face, misaligned eyes, bad mouth shape, extra limbs on human, fused fingers, broken hands  unnatural pose, stiff movement, floating subjects, wrong perspective, warped proportions, stretched cat, tiny head, giant head  dead fish look, deformed fish, unrealistic fish scales, melted texture, broken highlights, flat metal reflections  poor lighting, flat lighting, blown highlights, crushed shadows, incorrect shadows, inconsistent light direction  bad composition, off-center subject, awkward framing, cut off cat, cut off face, crowded scene, cluttered background, blocking foreground  low realism, cartoon, anime, illustration, painting, CGI look, game graphics, toy-like, plush-like fur  wrong setting, non-Dutch market, incorrect environment, indoor market, modern mall, supermarket aisles  bad emotion, neutral face, blank expression, lifeless eyes, no urgency  text, watermark, logo, captions, frame, borders'  # @param {type: "string"}

aspect_ratio ="720x1280 ( 9:16 )"  # @param [ "1024x1024 ( 1:1 )", "1152x896 ( 9:7 )", "896x1152 ( 7:9 )", "1152x864 ( 4:3 )", "864x1152 ( 3:4 )", "1248x832 ( 3:2 )", "832x1248 ( 2:3 )", "1280x720 ( 16:9 )", "720x1280 ( 9:16 )", "1344x576 ( 21:9 )", "576x1344 ( 9:21 )"]
seed = 0  # @param {type: "number"}
steps = 9  # @param {type: "number"}
cfg = 1.0  # @param {type: "number"}
denoise = 1.0  # @param {type: "number"}
Download_Image = False  # @param {type: "boolean"}

batch_size=1
sampler_name="euler"
scheduler="simple"
width,height = [int(x) for x in aspect_ratio.split("(")[0].strip().split("x")]

input = {
  "input": {
    "positive_prompt": positive_prompt,
    "negative_prompt": negative_prompt,
    "width": width,
    "height": height,
    "batch_size": batch_size,
    "seed": seed,
    "steps": steps,
    "cfg": cfg,
    "sampler_name": sampler_name,
    "scheduler": scheduler,
    "denoise": denoise,
  }
}

image_path,seed= generate(input)
img = Image.open(image_path)
print(f"\033[38;2;66;133;244mImage saved at {image_path}\033[0m")

if Download_Image:
  from google.colab import files
  files.download(image_path)
preview = img.copy()
preview.thumbnail((300, 300))
from IPython.display import display
#colab friendly
display(preview)

#original image
# display(img)




In [ ]:
#@title Display Original Image
display(img)

In [ ]:
#@title Download Image
from google.colab import files
files.download(image_path)